In [1]:
import os
import PIL
import numpy as np
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras import layers, models, optimizers, datasets, regularizers, callbacks
from keras_lr_finder import LRFinder
import matplotlib.pyplot as plt
%matplotlib inline
import re
import sklearn.metrics as skmetrics
import tensorflow as tf
import imgaug.augmenters as iaa
import imgaug as ia
from IPython.display import SVG
from sklearn.utils import shuffle

os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"

In [3]:
# Load CIFAR-100 dataset. Split between training and testing sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data()

class_count = len(np.unique(y_train))

y_train, y_test = to_categorical(y_train, class_count), to_categorical(y_test, class_count)

# Convert to float and normalize test images.
# Training images are normalized during augmentation
x_test = x_test.astype(np.float32)
x_test /= 255

AttributeError: module 'keras.datasets' has no attribute 'cifar100'

In [4]:
T_list = [
    iaa.Fliplr(p=0.5),
    iaa.Flipud(p=0.5),
    iaa.SaltAndPepper(p=(0.03, 0.2)),
    iaa.Affine(scale=(0.8, 1.2)),
    iaa.Affine(translate_px=(-8, 8)),
    iaa.Affine(rotate=(-90, 90)),
    iaa.Affine(shear=(-25, 25)),
    iaa.Pad(percent=(0.05, 0.20)),
]

img_augment = iaa.SomeOf(1, children=T_list, random_order=True)

def cutout(img, h=32, w=32, length=16):
    if len(img.shape) == 3:
        img = np.expand_dims(img, 0)
    imgs = np.zeros((len(img), h, w, 3))
    for i in range(img.shape[0]):
        mask = np.ones((h, w), dtype=np.float32)
        x = np.random.randint(w)
        y = np.random.randint(h)

        x1 = int(np.clip(x - length/2, 0, w))
        x2 = int(np.clip(x + length/2, 0, w))
        y1 = int(np.clip(y - length/2, 0, h))
        y2 = int(np.clip(y + length/2, 0, h))
        mask[y1:y2, x1:x2] = 0
        mask = np.expand_dims(mask, -1)
        imgs[i] = (img[i] * mask)
